In [1]:
import os 

In [2]:
os.getcwd()

'd:\\Learnings\\GenAI\\KNAC\\myfiles\\projects\\Agentic_Trading_Chatbot\\notebook'

In [3]:
os.chdir("..")

In [4]:
os.getcwd()

'd:\\Learnings\\GenAI\\KNAC\\myfiles\\projects\\Agentic_Trading_Chatbot'

# contants

In [ ]:
from dataclasses import dataclass 
from trading_bot.utils import load_yaml


CONFIG=load_yaml("config/config.yaml")

@dataclass 
class AgentsConstants:
    GOOGLE_REASONING_MODEL_NAME=CONFIG.MODELS.REASONERS.GEMINI_FLASH
    GROQ_REASONING_MODEL_NAME=CONFIG.MODELS.REASONERS.DEEPSEEK_R1

d:\Learnings\GenAI\KNAC\myfiles\projects\Agentic_Trading_Chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print("GOOGLE_REASONING_MODEL_NAME:", AgentsConstants.GOOGLE_REASONING_MODEL_NAME)
print("GROQ_REASONING_MODEL_NAME:", AgentsConstants.GROQ_REASONING_MODEL_NAME)

GOOGLE_REASONING_MODEL_NAME: gemini-2.5-pro-exp-03-25
GROQ_REASONING_MODEL_NAME: deepseek-r1-distill-llama-70b


# entity

In [7]:
from dataclasses import dataclass 


@dataclass 
class Agents:
    GOOGLE_REASONING_MODEL_NAME:str
    GROQ_REASONING_MODEL_NAME:str

# configuration

In [8]:
from dataclasses import dataclass 


@dataclass 
class AgentsConfig:
    GOOGLE_REASONING_MODEL_NAME=str(AgentsConstants.GOOGLE_REASONING_MODEL_NAME)
    GROQ_REASONING_MODEL_NAME=str(AgentsConstants.GROQ_REASONING_MODEL_NAME)

In [9]:
print("GOOGLE_REASONING_MODEL_NAME:", AgentsConfig.GOOGLE_REASONING_MODEL_NAME)
print(type(AgentsConfig.GOOGLE_REASONING_MODEL_NAME))
print("GROQ_REASONING_MODEL_NAME:", AgentsConfig.GROQ_REASONING_MODEL_NAME)
print(type(AgentsConfig.GROQ_REASONING_MODEL_NAME))

GOOGLE_REASONING_MODEL_NAME: gemini-2.5-pro-exp-03-25
<class 'str'>
GROQ_REASONING_MODEL_NAME: deepseek-r1-distill-llama-70b
<class 'str'>


# schema

In [12]:
from typing_extensions import Annotated, TypedDict
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

# components

In [ ]:
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from trading_bot.exception import CustomException
from langgraph.graph import StateGraph, START
from trading_bot.utils import load_reasoner
from trading_bot.logger import logging
from langchain_core.tools import tool
from dataclasses import dataclass , field
from typing import List
import sys 



@dataclass 
class AgentsComponents:
    __Agents_config:Agents
    __tools:List[tool]
    __graph:StateGraph = field(init=False)
    

    def __chatbot_node(self, state:State) -> dict:
        """node for llm chat

        Args:
            state (State): should have attribute/key named as messages, messages should be list

        Returns:
            dict: {"messages": [output of llm]}
        """
        try:
            logging.info("In get_chat_node")
            llm=load_reasoner(self.__Agents_config.GOOGLE_REASONING_MODEL_NAME, self.__Agents_config.GROQ_REASONING_MODEL_NAME)
            try:
                llm_name=llm.model
            except:
                llm_name=llm.model_name
            logging.info(f"Reasoner loaded {{{llm_name}}}")

            self.llm_with_tools=llm.bind_tools(self.__tools)
            logging.info("Out get_chat_node")
            return {"messages": [self.llm_with_tools.invoke(state["messages"])]}
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)

    def __post_init__(self) -> None:
        try:
            logging.info("In build_workflow")
            graph_builder=StateGraph(State)
        
            graph_builder.add_node("chatbot", self.__chatbot_node)
            
            tool_node=ToolNode(tools=self.__tools)
            graph_builder.add_node("tools", tool_node)
            
            graph_builder.add_conditional_edges("chatbot", tools_condition)
            graph_builder.add_edge("tools", "chatbot")
            graph_builder.add_edge(START, "chatbot")
            
            self.__graph=graph_builder.compile() 
            logging.info("Out build_workflow")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)
        
    def get_graph(self) -> StateGraph:
        """builds agentic graph/workflow

        Returns:
            StateGraph: graph/workflow
        """
        return self.__graph